In [ ]:
!pip install matplotlib

In [40]:
import pvlib
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# ======================
# === USER INPUTS ======
# ======================
lat = 36.42        # latitude (+N)
lon = -95.68        # longitude (+E; west is negative)
tz = "America/Los_Angeles"  # local timezone
start_year = 2015
end_year = 2024

# standard arrangement (fixed-tilt, south-facing)
system_dc_kw = 15.7 * 1000         # dc nameplate (kW)
dc_ac_ratio = 1.2          # dc/ac 
surface_azimuth = 180      # 180° = south-facing in N. Hemisphere
surface_tilt_deg = None    # if None, defaults to |lat|
albedo = 0.20              # typical for ground/roof
gamma_pdc = -0.004         # pvwatts temperature coefficient (1/°C)


In [ ]:
import pandas as pd
import pvlib

def simulate_pv_hourly(
    lat: float,
    lon: float,
    tz: str = "America/Boise",
    start_date: str = "2015-01-01",
    end_date: str = "2024-12-31",
    system_dc_kw: float = 6.0,
    dc_ac_ratio: float = 1.2,
    surface_tilt_deg: float = None,
    surface_azimuth: float = 180,
    albedo: float = 0.2,
    gamma_pdc: float = -0.004,
    net_efficiency: float = 0.90  # simple overall efficiency factor
) -> pd.DataFrame:
    """
    Simulate hourly PV generation (kW) using NASA POWER weather data and pvlib PVWatts.

    Parameters
    ----------
    lat, lon : float
        Site latitude and longitude (decimal degrees).
    tz : str
        Timezone string (e.g. 'America/Boise').
    start_date, end_date : str
        Simulation date range in 'YYYY-MM-DD' format.
    system_dc_kw : float
        DC nameplate capacity (kW).
    dc_ac_ratio : float
        DC/AC sizing ratio.
    surface_tilt_deg : float or None
        Array tilt (deg). If None, uses |lat| * 0.9.
    surface_azimuth : float
        Array azimuth (180 = south).
    albedo : float
        Ground reflectance.
    gamma_pdc : float
        PVWatts temperature coefficient (1/°C).
    net_efficiency : float
        Multiplier applied to AC output to represent total system efficiency (e.g., 0.90 = 90%).

    Returns
    -------
    pandas.DataFrame
        Hourly PV generation in kW (column 'ac_kw') indexed by timestamp.
    """

    if surface_tilt_deg is None:
        surface_tilt_deg = abs(lat) * 0.9

    # build location and PV system
    site = pvlib.location.Location(lat, lon, tz, name="site")
    inv_pdc0_kw = system_dc_kw / dc_ac_ratio
    inv_pdc0_w = inv_pdc0_kw * 1000

    system = pvlib.pvsystem.PVSystem(
        surface_tilt=surface_tilt_deg,
        surface_azimuth=surface_azimuth,
        albedo=albedo,
        module_parameters={"pdc0": system_dc_kw * 1000, "gamma_pdc": gamma_pdc},
        inverter_parameters={"pdc0": inv_pdc0_w, "eta_inv_nom": 0.96, "eta_inv_ref": 0.9637},
        temperature_model_parameters=pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS["sapm"]["open_rack_glass_glass"],
    )

    modelchain = pvlib.modelchain.ModelChain(
        system=system,
        location=site,
        aoi_model="physical",
        spectral_model="no_loss",
        transposition_model="haydavies",
        dc_model="pvwatts",
        ac_model="pvwatts",
        losses_model="no_loss",  # disable built-in PVWatts losses
    )

    # fetch NASA POWER data
    start = pd.Timestamp(start_date, tz="UTC")
    end = pd.Timestamp(end_date, tz="UTC")
    data, meta = pvlib.iotools.get_nasa_power(
        lat, lon,
        start=start, end=end,
        parameters=["ghi", "dhi", "dni", "T2M", "WS10M"],
    )

    # rename and handle timezone safely
    data = data.rename(columns={"T2M": "temp_air", "WS10M": "wind_speed"})
    try:
        data.index = data.index.tz_localize("UTC").tz_convert(tz)
    except TypeError:
        data.index = data.index.tz_convert(tz)

    weather = data[["ghi", "dni", "dhi", "temp_air", "wind_speed"]].copy()

    # run model
    modelchain.run_model(weather=weather)

    # hourly AC power (kW), scaled by net efficiency
    hourly = (modelchain.results.ac.fillna(0) / 1000.0 * net_efficiency).rename("ac_kw").to_frame()

    return hourly


In [ ]:
# df = simulate_pv_hourly(
#     lat=43.492,
#     lon=-112.040,
#     tz="America/Boise",
#     start_date="2024-01-01",
#     end_date="2024-12-31",
#     system_dc_kw=6.0,
# )

# df

In [73]:
def simulate_pv_hourly_multi_year(
    lat: float,
    lon: float,
    tz: str = "America/Boise",
    start_year: int = 2015,
    end_year: int = 2024,
    **kwargs
) -> pd.DataFrame:
    """
    Run simulate_pv_hourly() year by year and concatenate results.

    Parameters
    ----------
    lat, lon : float
        Coordinates.
    tz : str
        Timezone string.
    start_year, end_year : int
        Inclusive range of years to simulate.
    **kwargs :
        Any additional parameters accepted by simulate_pv_hourly().

    Returns
    -------
    pandas.DataFrame
        Combined hourly PV generation (kW) for all years.
    """
    frames = []

    for yr in range(start_year, end_year + 1):
        start_date = f"{yr}-01-01"
        end_date = f"{yr}-12-31"
        print(f"Running NASA POWER for {yr} ...")

        hourly = simulate_pv_hourly(
            lat=lat,
            lon=lon,
            tz=tz,
            start_date=start_date,
            end_date=end_date,
            **kwargs
        )
        frames.append(hourly)

    df = pd.concat(frames).sort_index()
    return df


In [74]:
df = simulate_pv_hourly_multi_year(
        lat=43.492,
        lon=-112.040,
        tz="America/Boise",
        start_year=2015,
        end_year=2024,
        system_dc_kw=6.0,
        dc_ac_ratio=1.2,
        net_efficiency=0.90,
    )

Running NASA POWER for 2015 ...
Running NASA POWER for 2016 ...
Running NASA POWER for 2017 ...
Running NASA POWER for 2018 ...
Running NASA POWER for 2019 ...
Running NASA POWER for 2020 ...
Running NASA POWER for 2021 ...
Running NASA POWER for 2022 ...
Running NASA POWER for 2023 ...
Running NASA POWER for 2024 ...


In [75]:
df

,ac_kw
2014-12-31 17:00:00-07:00,0.000000
2014-12-31 18:00:00-07:00,0.000000
2014-12-31 19:00:00-07:00,0.000000
2014-12-31 20:00:00-07:00,0.000000
2014-12-31 21:00:00-07:00,0.000000
...,...
2024-12-31 12:00:00-07:00,4.320000
2024-12-31 13:00:00-07:00,4.320000
2024-12-31 14:00:00-07:00,4.228387
2024-12-31 15:00:00-07:00,3.165385
